In [2]:
import sys
import os
print("\n".join(sys.path))


C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none\python310.zip
C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none\DLLs
C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none\lib
C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none
d:\Projects\AI_Trip_Planner\env

d:\Projects\AI_Trip_Planner\env\lib\site-packages
d:\Projects\AI_Trip_Planner\env\lib\site-packages\win32
d:\Projects\AI_Trip_Planner\env\lib\site-packages\win32\lib
d:\Projects\AI_Trip_Planner\env\lib\site-packages\Pythonwin


In [24]:
# Imports
import requests
import json

# Define the class
class WeatherForecastTool:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5"

    def get_current_weather(self, place: str):
        """Get current weather of a place"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                "q": place,
                "appid": self.api_key,
                "units" : "metric"  # use "imperial" for Fahrenheit
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            raise e

    def get_forecast_weather(self, place: str):
        """Get weather forecast of a place"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                "q": place,
                "appid": self.api_key,
                "cnt": 10,          # number of forecast entries
                "units": "metric"   # use "imperial" for Fahrenheit
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            raise e


In [25]:
import dotenv
import os
dotenv.load_dotenv()

# Create an instance
api_key=os.getenv("OPENWEATHER_API_KEY")
weather_tool = WeatherForecastTool(api_key=api_key)

# Test current weather
city = "Datia"
print("Current Weather:")
current = weather_tool.get_current_weather(city)
if current:
    print(f"Temperature: {current['main']['temp']} °C")
    print(f"Weather: {current['weather'][0]['description']}")
else:
    print("Failed to fetch current weather.")

# Test forecast weather
print("\nForecast Weather:")
forecast = weather_tool.get_forecast_weather(city)
if forecast:
    for entry in forecast["list"]:
        print(f"{entry['dt_txt']}: {entry['main']['temp']}°C, {entry['weather'][0]['description']}")
else:
    print("Failed to fetch forecast weather.")


Current Weather:
Temperature: 30.54 °C
Weather: moderate rain

Forecast Weather:
2025-07-06 21:00:00: 30.95°C, overcast clouds
2025-07-07 00:00:00: 30.03°C, light rain
2025-07-07 03:00:00: 32.07°C, light rain
2025-07-07 06:00:00: 34.67°C, light rain
2025-07-07 09:00:00: 35.19°C, light rain
2025-07-07 12:00:00: 35.81°C, broken clouds
2025-07-07 15:00:00: 32.58°C, overcast clouds
2025-07-07 18:00:00: 31.43°C, overcast clouds
2025-07-07 21:00:00: 30.28°C, light rain
2025-07-08 00:00:00: 29.76°C, moderate rain


In [26]:
import requests
import os
import dotenv
dotenv.load_dotenv()

api_key = os.getenv("EXCHANGE_RATE_API_KEY")

class CurrencyConverter:
    def __init__(self, api_key: str):
        self.base_url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/"
    
    def convert(self, amount:float, from_currency:str, to_currency:str):
        """Convert the amount from one currency to another"""
        url = f"{self.base_url}/{from_currency}"
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception("API call failed:", response.json())
        rates = response.json()["conversion_rates"]
        if to_currency not in rates:
            raise ValueError(f"{to_currency} not found in exchange rates.")
        return amount * rates[to_currency]

In [30]:
# Import your class  # Adjust the path if it's in utils/

# Load API key from .env
import os
import dotenv
dotenv.load_dotenv()
api_key = os.getenv("EXCHANGE_RATE_API_KEY")

# Create an instance
converter = CurrencyConverter(api_key)

# Test conversion
try:
    amount = 10
    from_currency = "USD"
    to_currency = "Russian"
    
    converted = converter.convert(amount, from_currency, to_currency)
    print(f"{amount} {from_currency} = {converted:.2f} {to_currency}")
except Exception as e:
    print("❌ Error during conversion:", e)


❌ Error during conversion: Russian not found in exchange rates.


In [12]:
import os
import dotenv
from langchain_tavily import TavilySearch

dotenv.load_dotenv()

class TavilyPlaceSearchTool:
    def __init__(self):
        self.api_key = os.getenv("TAVILY_API_KEY")
        if not self.api_key:
            raise ValueError("TAVILY_API_KEY is missing in environment variables.")

    def _get_tavily_tool(self):
        return TavilySearch(
            topic="general",
            include_answer="advanced",
            tavily_api_key=self.api_key  # ✅ Inject API Key directly
        )

    def _safe_invoke(self, query: str) -> str:
        """
        Internal helper to invoke Tavily safely and return answer or fallback.
        """
        try:
            tavily_tool = self._get_tavily_tool()
            result = tavily_tool.invoke({"query": query})
            return result.get("answer", str(result))
        except Exception as e:
            return f"⚠️ Failed to retrieve info: {e}"

    def search_attractions(self, place: str) -> str:
        query = (
            f"As a travel planner, give me the top tourist attractions in and around {place}, "
            f"with descriptions, reasons for popularity, and any ticket or timing info."
        )
        return self._safe_invoke(query)

    def search_veg_restaurants(self, place: str) -> str:
        query = (
            f"List the best vegetarian restaurants in and around {place}. "
            f"Include local dishes they are known for and user recommendations."
        )
        return self._safe_invoke(query)

    def search_nonveg_restaurants(self, place: str) -> str:
        query = (
            f"What are the best non-vegetarian restaurants in and around {place}? "
            f"Mention top dishes, cuisines, and chef specialties if any."
        )
        return self._safe_invoke(query)

    def search_activities(self, place: str) -> str:
        query = (
            f"What are the best cultural, adventure, and local activities that a traveler must try in {place}?"
        )
        return self._safe_invoke(query)

    def search_transportation(self, place: str) -> str:
        query = (
            f"What are the different modes of transportation in {place} for tourists, "
            f"including local transit, cabs, and long-distance travel options?"
        )
        return self._safe_invoke(query)


In [13]:
# test_tavily_tool.py

# Create an instance
tool = TavilyPlaceSearchTool()

# Define a place to test
place = "Udaipur"

print("\n📍 Top Attractions in", place)
print(tool.search_attractions(place))

print("\n🥗 Best Vegetarian Restaurants in", place)
print(tool.search_veg_restaurants(place))

print("\n🍗 Best Non-Vegetarian Restaurants in", place)
print(tool.search_nonveg_restaurants(place))

print("\n🎉 Top Activities to Try in", place)
print(tool.search_activities(place))

print("\n🚕 Transportation Options in", place)
print(tool.search_transportation(place))



📍 Top Attractions in Udaipur
Udaipur, known as the "City of Lakes" and often called the "Venice of the East," offers numerous spectacular tourist attractions that draw visitors from around the world. The City Palace stands as the crown jewel, showcasing magnificent Rajasthani architecture and offering panoramic views of the city and lakes. Lake Pichola provides enchanting boat rides with views of the iconic Lake Palace and Jag Mandir Island, while Ambrai Ghat on its banks offers serene sunset views perfect for photography. The Jagdish Temple represents beautiful Hindu architecture and spiritual significance, while Fateh Sagar Lake provides another scenic water body for boating and relaxation. For those seeking spiritual experiences, Eklingji Mahadev Temple and Karni Mata Temple offer religious significance and architectural beauty. Adventure seekers can take day trips to nearby Kumbhalgarh Fort, famous for having the second-longest wall in the world, and the stunning Ranakpur Jain Tem